# Проект "Бот с картинками", на основе собранных данных


## Асинхронность (необязательный раздел)

### Синхронный код

Синхронный код — это тот, к которому мы уже привыкли. Он выполняется строчка за строчкой. Каждая команда ждет завершения предыдущей, прежде чем перейти к следующей.

#### Пример:

Когда вы пишете код:

```python
print("Первое сообщение")
print("Второе сообщение")
print("Третье сообщение")
```

Программа сначала выведет "Первое сообщение", затем "Второе", а потом "Третье". Ничего не происходит параллельно — всё строго по порядку.

#### Почему это удобно?

- Простота: Легко понимать и отлаживать, так как всё выполняется последовательно.
- Привычка: Мы привыкли мыслить именно так, и это естественно для большинства задач.

#### Когда это может быть проблемой?

Если одна из операций занимает много времени — например, чтение данных с жёсткого диска или загрузка из интернета — программа будет "зависать" и не сможет выполнять другие задачи. Это может быть неудобно для пользователя.

В таких случаях стоит задуматься об асинхронности, чтобы сделать работу программы более быстрой и отзывчивой.

### Решение: асинхронность

Асинхронный код позволяет программе одновременно отправлять запросы на диск или в интернет и продолжать работать, пока ждёт ответа. Программа может "раздать" задания и затем дождаться, когда все они будут выполнены, что делает её более быстрой и отзывчивой.


## 1. Загрузка изображений из поисковой системы bing.com


In [7]:
import base64
import mimetypes
import urllib.parse
import os
import asyncio
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


OUTPUT_DIR = "images"
MAX_TIMEOUT = 2  # Продолжительность ожидания загрузки изображений (в секундах)

# Создаем папку для изображений, если она не существует
os.makedirs(OUTPUT_DIR, exist_ok=True)


def download_image(url: str, i: int) -> None:
    """
    Скачивает изображение по указанному URL.

    Если URL содержит данные в формате base64, изображение будет
    декодировано и сохранено локально. В противном случае,
    выполняется HTTP-запрос для получения изображения.

    :param url: URL изображения или данные в формате base64.
    :param i: Индекс изображения, используемый для формирования имени файла.
    """
    if "base64" in url:
        # Обработка base64 изображений
        mime_type = url.split(";")[0].split(":")[1]
        content = base64.b64decode(url.split(",")[1])
        ext = mimetypes.guess_extension(mime_type)
        if ext and "svg" not in ext:
            fname = os.path.join(OUTPUT_DIR, f"{i}{ext}")
            with open(fname, "wb") as f:
                f.write(content)
            # Скачали {fname} из base64
    else:
        # Обработка обычных URL изображений
        try:
            response = requests.get(url)
            if response.status_code == 200:
                mime_type = response.headers.get("content-type")
                ext = mimetypes.guess_extension(mime_type)
                if ext and "svg" not in ext:
                    fname = os.path.join(OUTPUT_DIR, f"{i}{ext}")
                    with open(fname, "wb") as f:
                        f.write(response.content)
                    # Скачали {fname} из URL
            else:
                print(f"Не удалось скачать изображение: {url}, статус: {response.status_code}")
        except Exception as e:
            print(f"Ошибка при скачивании {url}: {e}")


async def main() -> None:
    """
    Основная асинхронная функция для инициализации веб-драйвера,
    выполнения поиска изображений и их скачивания.

    Использует Selenium для прокрутки страницы и извлечения URL изображений.
    """
    driver = webdriver.Chrome()
    searches = ["котики", "щеночки"]

    srcs = []
    try:
        for search in searches:
            q = urllib.parse.quote_plus(search)
            url = f"https://www.bing.com/images/search?q={q}&form=RESTAB&first=1"
            driver.get(url)

            # Условное ожидание: ждём загрузки изображений
            WebDriverWait(driver, MAX_TIMEOUT).until(EC.presence_of_all_elements_located((By.XPATH, "//img")))

            # Прокрутка страницы и сбор изображений
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                # Прокручиваем страницу вниз с помощью JavaScript
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                try:
                    # Условное ожидание: ждем, пока появится хотя бы одно новое изображение
                    WebDriverWait(driver, MAX_TIMEOUT).until(lambda d: len(d.find_elements(By.XPATH, "//img")) > len(srcs))
                except TimeoutException:
                    print(f"[{search}]: Время ожидания истекло, новых изображений не найдено.")
                    break  # Выход из цикла, если время ожидания истекло

                # Получаем новые изображения
                imgs = driver.find_elements(By.XPATH, "//img")
                for img in imgs:
                    src = img.get_attribute("src")
                    if src:
                        srcs.append(src)  # Добавляем только уникальные ссылки

                # Проверяем высоту страницы
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:  # Если высота не изменилась, выходим
                    print(f"[{search}]: Высота прокрутки не изменилась.")
                    break
                last_height = new_height

        print(f"Найдено изображений: {len(srcs)}")

        # Асинхронное скачивание изображений
        tasks = [asyncio.to_thread(download_image, url, i) for i, url in enumerate(srcs)]
        await asyncio.gather(*tasks)
    finally:
        # Закрытие драйвера
        driver.quit()


# Запуск асинхронного главного метода
await main()

[котики]: Время ожидания истекло, новых изображений не найдено.
[щеночки]: Время ожидания истекло, новых изображений не найдено.
Найдено изображений: 561
Ошибка при скачивании https://th.bing.com/th?q=%d0%9a%d0%b0%d1%80%d1%82%d0%b8%d0%bd%d0%ba%d0%b8+%d0%9d%d0%b0+%d0%a0%d0%b0%d0%b1%d0%be%d1%87%d0%b8%d0%b9+%d0%a1%d1%82%d0%be%d0%bb+%d0%9a%d0%be%d1%82%d0%b8%d0%ba%d0%b8&w=120&h=120&c=1&rs=1&qlt=90&cb=1&pid=InlineBlock&mkt=en-WW&cc=BY&setlang=ru&adlt=moderate&t=1&mw=247: HTTPSConnectionPool(host='th.bing.com', port=443): Max retries exceeded with url: /th?q=%D0%9A%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8+%D0%9D%D0%B0+%D0%A0%D0%B0%D0%B1%D0%BE%D1%87%D0%B8%D0%B9+%D0%A1%D1%82%D0%BE%D0%BB+%D0%9A%D0%BE%D1%82%D0%B8%D0%BA%D0%B8&w=120&h=120&c=1&rs=1&qlt=90&cb=1&pid=InlineBlock&mkt=en-WW&cc=BY&setlang=ru&adlt=moderate&t=1&mw=247 (Caused by SSLError(SSLError(1, '[SSL] record layer failure (_ssl.c:1000)')))
Ошибка при скачивании https://th.bing.com/th?id=OIF.37iKVssiE6sTygmhFRKQ%2fA&w=295&h=187&c=7&r=0

# Асинхронность в этом коде

В этом коде асинхронность используется для улучшения производительности при скачивании изображений. Давайте разберём, как это работает.

## Основные моменты:

1. **Функция `download_image`:**

   - Эта функция отвечает за скачивание изображений.
   - В синхронном коде эта функция будет блокировать выполнение, пока изображение не будет скачано.

2. **Асинхронная функция `main`:**

   - В этой функции происходит поиск изображений с помощью Selenium и их последующее скачивание.
   - Асинхронность начинается, когда мы собираем URL-адреса изображений в списке `srcs`.

3. **Использование `asyncio.to_thread`:**

   - Вместо того чтобы ждать завершения каждой загрузки, мы создаём **список задач** `tasks` и запускаем их.

4. **`await asyncio.gather(*tasks)`:**
   - Здесь мы дожидаемся, пока все задачи в списке `tasks` завершатся.

## Почему это важно?

- **Эффективность:** Асинхронный подход позволяет одновременно скачивать несколько изображений (это значительно сокращает общее время выполнения, особенно если у вас много изображений).
- **Отзывчивость:** Эта технология оставляет программу остаётся отзывчивой, не "зависая" на каждой загрузке. Это особенно важно, когда взаимодействуешь с пользователем.
